In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from es.utils.config_loaders import Config 
from es.dataloader.loader import HighResWindSpeed
from es.backbone import KarrasUnet
from es.diffusion import LElucidateDiffusion

from lightning import Trainer

from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch.callbacks import LearningRateMonitor



In [4]:
config = Config(system="Mohit").get_config()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


wandb: Currently logged in as: itsmohitanand (earth-ai). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/mila/m/mohit.anand/.netrc


In [5]:
dl = HighResWindSpeed(config)
dl.setup(stage="train")

Scaling params found

In [5]:
backbone_params = config.experiment.model.backbone.params.toDict()
diffusion_config = config.experiment.model.diffusion
diffusion_config

Munch({'name': 'ElucidateDiffusion', 'optimizer': Munch({'params': Munch({'name': 'Adam', 'lr': '3e-3', 'T_max': 200000, 'eta_min': '3e-5'})}), 'model': Munch({'params': Munch({'num_sample_steps': 32, 'self_conditioning': False})}), 'loss_mask': None, 'latitude_weight': None, 'variable_weight': None})

In [6]:
backbone = KarrasUnet(**backbone_params)
diffusion = LElucidateDiffusion(backbone, diffusion_config)

In [7]:
wandb_project =  "test_hrws"
wandb_entity = "itsmohitanand"
wandb_logger = WandbLogger(project=wandb_project, entity=wandb_entity, experiment=None)
wandb_logger.experiment.config.update(config.toDict())


wandb: Currently logged in as: itsmohitanand. Use `wandb login --relogin` to force relogin


CommError: failed to upsert bucket: returned error 403 Forbidden: {"errors":[{"message":"permission denied","path":["upsertBucket"],"extensions":{"code":"PERMISSION_ERROR"}}],"data":{"upsertBucket":null}}

In [9]:
lr_monitor = LearningRateMonitor(logging_interval='step')


In [10]:
trainer = Trainer(accelerator="gpu", logger=wandb_logger, callbacks=[lr_monitor])
trainer.fit(model=diffusion, datamodule=dl)

/home/mila/m/mohit.anand/miniforge3/envs/forest_mssl/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/mila/m/mohit.anand/miniforge3/envs/forest_mssl ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


/home/mila/m/mohit.anand/miniforge3/envs/forest_mssl/lib/python3.11/site-packages/lightning/pytorch/loops/utilities.py:73: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.


CommError: failed to upsert bucket: returned error 403 Forbidden: {"errors":[{"message":"permission denied","path":["upsertBucket"],"extensions":{"code":"PERMISSION_ERROR"}}],"data":{"upsertBucket":null}}

In [10]:
train_dl = dl.train_dataloader()

Temporal check passed

In [12]:
for x, y in train_dl:
    break

In [13]:
x.shape

torch.Size([4, 3, 720, 1440])

In [14]:
y.shape

torch.Size([4, 2, 720, 1440])